In [23]:
function ej(n,i)
    v = zeros(n)
    v[i] = 1.0
    return v
end

function pvec2mat(p)
    n = length(p)
    P = zeros(n,n)
    for i in 1:n
        P[i,:] = ej(n,p[i])
    end
    return sparse(P)
end

"""
    [L,U,Q] = luq(A,tol)
Calculate the following decomposition
A = L |Ubar  0 | Q
      |0     0 |
where Ubar is a square invertible matrix
and matrices L, Q are invertible.
Based on lu decomposition.

Inputs:
* `A`: a sparse matrix
* `tol`: used to separate zero and nonzero values

Note: Julia's lufact always does column pivoting
for sparse matrices.

Ported from Pawel Kowal:
Copyright  (c) Pawel Kowal (2006)
All rights reserved
LREM_SOLVE toolbox is available free for noncommercial academic use only.
pkowal3@sgh.waw.pl
"""
function luq(A,tol)
    A = sparse(A)
    m,n = size(A)

    ###################
    ## Special cases ##
    ###################
    if m == 0 || n == 0
        L = speye(m)
        U = A
        Q = speye(n)
        return L,U,Q
    end

    ######################
    ## LU decomposition ##
    ######################
    F = lufact(A)
    L,U,P,Q = F[:L],F[:U],pvec2mat(F[:p]),pvec2mat(F[:q])

    p = m - size(L,2)
    L = [P'*L P[m-p+1:m,:]']
    U = [U;spzeros(p,n)]

    ##########################
    ## Find zero pivot rows ##
    ##########################
    if size(U,1) == 1 || size(U,2) == 1
        S = U[1,1]
    else
        S = diag(U)
    end
    I = find(abs(S).>tol) # indices of nonzero diagonal elements of U
    Jl = setdiff(1:m,I) # indices of zero diagonal elements of U
    Jq = setdiff(1:n,I) # indices of zero diagonal elements of U

    Ubar1 = U[I,I] # axes of U with nonzero diagonals "invertible block"
    Ubar2 = U[Jl,Jq] # axes of U with zero diagonals
    Qbar1 = Q[I,:] # Rows of col permutation matrix corr. to nonzero diag. of U
    Lbar1 = L[:,I] # Cols of L corr. to nonzero diag. of U

    ##########################################
    ## Eliminate nonzero elements below and ##
    ## on right of invertible block of U,   ##
    ## and update L and Q                   ##
    ##########################################
    if !isempty(I)
        Utmp = U[I,Jq] # Nonzero rows, zero cols
        # Ubar1 square invertible. Re-write:
        X = (full(U[Jl,I])/Ubar1)'
        
        Ubar2 = Ubar2 - X'*Utmp
        Lbar1 = Lbar1 + L[:,Jl]*X'

        X = Ubar1\full(Utmp)
        Qbar1 = Qbar1 + X*Q[Jq,:]
    end
    
#     for idx in I
#         Utmp = U[idx,Jq] # zero cols of row idx; row vec
#         X = Ubar1'\(U[Jl,idx]')
#         Ubar2 -= X'*Utmp
#         Lbar1 += L[:,Jl]*X'
        
#         X = Ubar1\Utmp
#         Qbar1 += X*Q[Jq,:]
#     end

    ################################################
    ## Find rows and cols with only zero elements ##
    ################################################
    I2 = find(maxabs(Ubar2,2).>tol);
    I5 = find(maxabs(Ubar2,1).>tol);

    I3 = Jl[I2]
    I4 = Jq[I5]
    Jq = setdiff(Jq,I4)
    Jl = setdiff(Jl,I3)

    ########################################
    ## Find part of U which is not in the ##
    ## required form                      ##
    ########################################
    A = Ubar2[I2,I5]

    ####################################
    ## Perform luq decomposition on A ##
    ####################################
    L1,U1,Q1 = luq(A,tol)

    ##################
    ## Update L,U,Q ##
    ##################
    Lbar2 = L[:,I3]*L1
    Qbar2 = Q1*Q[I4,:]
    L = [Lbar1 Lbar2 L[:,Jl]]
    Q = [Qbar1; Qbar2; Q[Jq,:]]

    m1,m2,n2 = length(I), length(I3), length(I4)
    U = [Ubar1 spzeros(m1,n-m1);
        spzeros(m2,m1) U1 spzeros(m2,n-m1-n2);
        spzeros(m-m1-m2,n)]

    return L,U,Q
end

function spspaces(
    A,
    tol=max(maximum(size(A))*norm(A,1)*eps(),100*eps())
    )
    L,U,Q = luq(A,tol)

    if !isempty(Q)
        QQ = inv(Q)
    else
        QQ = Q
    end
    S = maxabs(U,1)
    I = find(S.>tol)
    if !isempty(S)
        J = find(S.<=tol)
    else
        J = (1:size(S,2))
    end
    return QQ[:,J]
end

spspaces (generic function with 2 methods)

In [6]:
F = lufact(sprandn(100,200,0.1))

UMFPACK LU Factorization of a 100-by-200 sparse matrix
Ptr{Void} @0x00000000038786b0


In [20]:
A = sprand(1000,2000,0.2)
x1 = @timed spspaces(A)[1];
x1

(-0.9147406153164831,4.527522398,777095840,0.113422534,Base.GC_Diff(777095840,2250,10,18575,3127,2179,113422534,26,3))

In [24]:
maxabs(A*spspaces(A))

7.994840900416023e-12

In [162]:
x2 = @timed kernel_rotation(A)[1];
x2

(0.013268974325458188,0.592955337,232505016,0.007243147,Base.GC_Diff(232505016,100,1,47,4,59,7243147,8,0))

In [122]:
function kernel_rotation(A::SparseMatrixCSC{Float64,Int64}; spqr=true)
    m,n = size(A)

    # Assume A always has full row rank of m.
    # It may be possible for this assumption to fail
    # due to numerics, but a rank() check is expensive.
    dim_N = n - m # dimension of nullspace of A

    if spqr
        F = qrfact(sparse(A'))
        # B selects last dim_N cols of Q:
        B = [zeros(size(A,2)-dim_N,dim_N); eye(dim_N)]
        N = sparse(SparseMatrix.SPQR.qmult(SparseMatrix.SPQR.QX, F, SparseMatrix.CHOLMOD.Dense(B)))
        return N
    else
        q = qr(A'; thin=false)[1]
        return q[:,end-dim_N+1:end]
    end
end

kernel_rotation (generic function with 1 method)

## Wikipedia method

[Wikipedia][1] has a simple algorithm for using Gaussian elimination to find a basis for the nullspace.

[1]: https://en.wikipedia.org/wiki/Kernel_(linear_algebra)#Computation_by_Gaussian_elimination

In [38]:
# A = sprandn(100,200,0.1)
A = [
    1 0 -3 0 2 -8;
    0 1 5 0 -1 4;
    0 0 0 1 7 -9;
    0 0 0 0 0 0
]
m,n = size(A)
AI = [A;speye(n)]

# perform column operations to place upper part
# in column echelon form:
for i in 1:m
    

Base.LinAlg.LU{Float64,Array{Float64,2}}(6x10 Array{Float64,2}:
 -8.0    4.0       -9.0       0.0  0.0  0.0  …  0.0   0.0        1.0      
  0.375  3.5        3.375     0.0  0.0  0.0     0.0   0.0       -0.375    
 -0.25   0.0        4.75      0.0  0.0  0.0     0.0   1.0        0.25     
 -0.0    0.0        0.210526  0.0  0.0  0.0     1.0  -0.210526  -0.0526316
 -0.125  0.142857  -0.338346  0.0  1.0  0.0     0.0   0.338346   0.263158 
 -0.0    0.285714  -0.203008  0.0  0.0  1.0  …  0.0   0.203008   0.157895 ,Int32[6,3,5,4,6,6],4)

In [61]:
?nnz

search: 

```
nnz(A)
```

Returns the number of stored (filled) elements in a sparse matrix.


nnz nonzeros findnz countnz



In [64]:
?spones

search: 

```
spones(S)
```

Create a sparse matrix with the same structure as that of `S`, but with every nonzero element having the value `1.0`.


spones unsafe_pointer_to_objref



In [65]:
?valence

search: 

LoadError: LoadError: "valence" is not defined in module Main
while loading In[65], in expression starting on line 119

InvalidStateException

Couldn't find valence
Perhaps you meant valtype or values


In [62]:
?findnz

search: 

```
findnz(A)
```

Return a tuple `(I, J, V)` where `I` and `J` are the row and column indexes of the non-zero values in matrix `A`, and `V` is a vector of the non-zero values.


findnz findn findnext findin findmin



In [42]:
F[:U]

6x10 Array{Float64,2}:
 -8.0  4.0  -9.0    0.0  0.0  0.0   0.0       0.0   0.0        1.0      
  0.0  3.5   3.375  0.0  0.0  0.0   1.0       0.0   0.0       -0.375    
  0.0  0.0   4.75   0.0  0.0  0.0   0.0       0.0   1.0        0.25     
  0.0  0.0   0.0    0.0  0.0  0.0   0.0       1.0  -0.210526  -0.0526316
  0.0  0.0   0.0    0.0  1.0  0.0  -0.142857  0.0   0.338346   0.263158 
  0.0  0.0   0.0    0.0  0.0  1.0  -0.285714  0.0   0.203008   0.157895 

In [59]:
L,U,p = lu(A')

(
6x4 Array{Float64,2}:
  1.0    0.0        0.0       0.0
  0.375  1.0        0.0       0.0
 -0.25   0.0        1.0       0.0
 -0.0    0.0        0.210526  1.0
 -0.0    0.285714  -0.203008  0.0
 -0.125  0.142857  -0.338346  0.0,

4x4 Array{Float64,2}:
 -8.0  4.0  -9.0    0.0
  0.0  3.5   3.375  0.0
  0.0  0.0   4.75   0.0
  0.0  0.0   0.0    0.0,

Int32[6,3,5,4,2,1])

In [51]:
eye(n)

6x6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [54]:
L'*eye(n)

4x6 Array{Float64,2}:
 1.0  0.375  -0.25  0.0        0.0       -0.125   
 0.0  1.0     0.0   0.0        0.285714   0.142857
 0.0  0.0     1.0   0.210526  -0.203008  -0.338346
 0.0  0.0     0.0   1.0        0.0        0.0     